In [1]:
import pandas as pd
pd.options.display.max_columns=1000
pd.options.display.width=200
pd.options.display.min_rows=60
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, RandomizedSearchCV

from datetime import datetime
import xgboost as xgb
from xgboost import XGBClassifier

# import custom functions from hyperparamfuncs.py
from hyperparamfuncs import *

In [2]:
# create empty hyperparameter table
hyperparam_table = []

In [3]:
# run a xgboost model on raw data before any data cleaning/feature engineering
filename = 'data/clinvar_conflicting.csv'
hyper_dict = {'test_size': 0.05, 
              'random_state': 0, 
              'data_size': None,
              'scaling': None,
              'filename': filename,
              'model': XGBClassifier,
              'class_weight': None
             }
clf, hyperparam_table = train_eval(filename, hyper_dict, hyperparam_table)

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (0,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/gksullan/Documents/Springboard/Capstone_Project_1/conflicting_geneticvariants/hyperparamfuncs.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=1, inplace=True)


In [4]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head()

,test_size,random_state,data_size,scaling,filename,model,class_weight,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,feature_importances
0,0.05,0,"(65188, 4)",None,data/clinvar_conflicting.csv,<class 'xgboost.sklearn.XGBClassifier'>,None,0.761656,0.79326,2275,147,630,208,0.348701,0.585915,0.24821,"[0.13841417, 0.11646682, 0.33226815, 0.4128508]"


In [5]:
# balance the class weights
filename = 'data/clinvar_conflicting.csv'
hyper_dict = {'test_size': 0.05, 
              'random_state': 0, 
              'data_size': None,
              'scaling': None,
              'filename': filename,
              'model': XGBClassifier,
              'class_weight': 'balanced'
             }
clf, hyperparam_table = train_eval(filename, hyper_dict, hyperparam_table)

In [6]:
# Use cleaned data file with NLP features
filename = 'data/data_cleanednlp4.csv'
hyper_dict = {'test_size': 0.05, 
              'random_state': 0, 
              'data_size': None,
              'scaling': None,
              'filename': filename,
              'model': XGBClassifier,
              'class_weight': None
             }
clf, hyperparam_table = train_eval(filename, hyper_dict, hyperparam_table)

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head()

,test_size,random_state,data_size,scaling,filename,model,class_weight,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,feature_importances
0,0.05,0,"(65188, 4)",None,data/clinvar_conflicting.csv,<class 'xgboost.sklearn.XGBClassifier'>,None,0.761656,0.793260,2275,147,630,208,0.348701,0.585915,0.248210,"[0.13841417, 0.11646682, 0.33226815, 0.4128508]"
1,0.05,0,"(65188, 4)",None,data/clinvar_conflicting.csv,<class 'xgboost.sklearn.XGBClassifier'>,balanced,0.761656,0.793260,2275,147,630,208,0.348701,0.585915,0.248210,"[0.13841417, 0.11646682, 0.33226815, 0.4128508]"
2,0.05,0,"(65188, 61)",None,data/data_cleanednlp4.csv,<class 'xgboost.sklearn.XGBClassifier'>,None,0.765031,0.818208,2227,195,571,267,0.410769,0.577922,0.318616,"[0.01426685, 0.022147033, 0.055169754, 0.07037..."


In [15]:
# Use RandomizedSearchCV to find the best hyperparameters for this model
filename = 'data/data_cleanednlp4.csv'
param_dist = {'learning_rate': [0.1, 0.15, 0.2, 0.25, 0.3],
              "max_depth": [5, 7, 10, None],
              "colsample_bytree": [0.5, 0.7, 0.9, 1.0],
              "n_estimators": [50, 100, 300, 500],
              "gamma": [0, 0.1, 0.2, 0.3, 0.4, 0.5]}
df = pd.read_csv(filename)
df, var_class = preprocess_data(df)
x_train, x_test, y_train, y_test = train_test_split(df, var_class,
                                                    test_size=hyper_dict['test_size'],
                                                    random_state=hyper_dict['random_state'])
# Instantiate a XGBoost classifier
clf = XGBClassifier(random_state=0)

# Instantiate the RandomizedSearchCV object
clf_cv = RandomizedSearchCV(clf, param_dist, scoring='f1', cv=2)

# Fit it to the data
clf_cv.fit(x_train, y_train)

/home/gksullan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (34,36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/gksullan/Documents/Springboard/Capstone_Project_1/conflicting_geneticvariants/hyperparamfuncs.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=1, inplace=True)


RandomizedSearchCV(cv=2, error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_co...
                                           validate_parameters=False,
                                           verbosity=None),
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions=

In [10]:
# preview the best parameters and score
print(f"Tuned XGBoost Parameters: {clf_cv.best_params_}")
print(f"Best score is {clf_cv.best_score_}")

Tuned XGBoost Parameters: {'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.25, 'gamma': 0.4, 'colsample_bytree': 0.9}
Best score is 0.6227965658695761


In [11]:
cv_params = clf_cv.best_params_

In [12]:
# Using this model with the best params, predict labels on test data
hyper_dict = {'test_size': 0.05, 
              'random_state': 0, 
              'data_size': str(df.shape),
              'scaling': None,
              'filename': filename,
              'model': XGBClassifier
             }
clf = XGBClassifier(random_state=0)
clf.set_params(**cv_params)
hyper_dict.update(cv_params)
hyperparam_table += [hyper_dict]
clf.fit(x_train, y_train)
pred = clf.predict(x_test)
score = clf.score(x_test,y_test)
hyperparam_table[-1]['test_score'] = score
training_score = clf.score(x_train,y_train)
hyperparam_table[-1]['train_score'] = training_score

tn, fp, fn, tp = confusion_matrix(y_test,pred).ravel()
hyperparam_table[-1]['tn'] = tn
hyperparam_table[-1]['fp'] = fp
hyperparam_table[-1]['fn'] = fn
hyperparam_table[-1]['tp'] = tp

f1 = f1_score(y_test,pred)
hyperparam_table[-1]['f1_score'] = f1
precision = precision_score(y_test,pred)
hyperparam_table[-1]['precision'] = precision
recall = recall_score(y_test,pred)
hyperparam_table[-1]['recall'] = recall

hyperparam_table[-1]['feature_importances'] = clf.feature_importances_

In [13]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head(10)

,test_size,random_state,data_size,scaling,filename,model,class_weight,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,feature_importances,n_estimators,max_depth,learning_rate,gamma,colsample_bytree
0,0.05,0,"(65188, 4)",None,data/clinvar_conflicting.csv,<class 'xgboost.sklearn.XGBClassifier'>,None,0.761656,0.793260,2275,147,630,208,0.348701,0.585915,0.248210,"[0.13841417, 0.11646682, 0.33226815, 0.4128508]",NaN,NaN,NaN,NaN,NaN
1,0.05,0,"(65188, 4)",None,data/clinvar_conflicting.csv,<class 'xgboost.sklearn.XGBClassifier'>,balanced,0.761656,0.793260,2275,147,630,208,0.348701,0.585915,0.248210,"[0.13841417, 0.11646682, 0.33226815, 0.4128508]",NaN,NaN,NaN,NaN,NaN
2,0.05,0,"(65188, 61)",None,data/data_cleanednlp4.csv,<class 'xgboost.sklearn.XGBClassifier'>,None,0.765031,0.818208,2227,195,571,267,0.410769,0.577922,0.318616,"[0.01426685, 0.022147033, 0.055169754, 0.07037...",NaN,NaN,NaN,NaN,NaN
3,0.05,0,"(65188, 61)",None,data/data_cleanednlp4.csv,<class 'xgboost.sklearn.XGBClassifier'>,NaN,0.769325,0.926221,2203,219,533,305,0.447871,0.582061,0.363962,"[0.015140971, 0.018364254, 0.024925346, 0.0488...",500.0,7.0,0.25,0.4,0.9


In [14]:
# save the hyperparameter table to a .pickle file
time = datetime.now().strftime("%Y%m%d_%H:%M:%S")
filename = 'hyperparameter_tables/hyperparameter_table'+time+'.pkl'
df_hyp.to_pickle(filename)